In [3]:
import numpy as np
from scipy.optimize import linear_sum_assignment
from itertools import combinations


class Assignment():

    robot_places = np.array([1, 1, 1, 6, 6, 6])
    picker_places = np.array([1, 3, 2, 6, 7, 9, 0])

    min_place = 0
    max_place = 2


    def search(self):
        n_robots = self.robot_places.size
        cs = list(combinations(range(self.min_place,self.max_place + 1), n_robots))
        for c in cs:
            self.robot_places = np.array(c)
            self.compute_cost_matrices()
            self.compute_assignment()
            self.show_allocation()

        

    def show_allocation(self):
        rows = self.assignment[0]
        cols = self.assignment[1]
        n_robots = self.robot_places.size
        for i in range(0, len(cols)):
            if cols[i] < n_robots:
                print(f'robot {cols[i]} serves picker {rows[i]}.')
            else:
                print(f'picker {rows[i]} is currently not served')
        c = self.cost[rows, cols].sum()
        print(f'total costs: {c}')
        return c

    def compute_cost_matrices(self):
        # repeat the positions as columns and rows to prepare for diff
        self.robot_places_mat = np.tile(self.robot_places, (self.picker_places.size,1))
        #robot_places_mat = np.append(robot_places_mat, np.zeros((picker_places.,)), axis=1)
        self.picker_places_mat = np.tile(self.picker_places, (self.robot_places.size,1)).transpose()

        #display('robot_places_mat: ', self.robot_places_mat)
        #display('picker_places_mat: ', self.picker_places_mat)

        # calculate costs simply as 
        self.cost = np.abs(self.picker_places_mat - self.robot_places_mat)

        # add zero costs for unassigned relations (make it square)
        self.cost = np.append(self.cost, np.zeros((self.picker_places.size,self.picker_places.size - self.robot_places.size)), axis=1)
        #display('costs (rows: pickers, columns: robots): ', self.cost)


    def compute_assignment(self):
        self.assignment = linear_sum_assignment(self.cost)




In [4]:
ac = Assignment()
ac.compute_cost_matrices()
ac.compute_assignment()
ac.show_allocation()
#ac.search()

robot 0 serves picker 0.
robot 5 serves picker 1.
robot 1 serves picker 2.
robot 3 serves picker 3.
robot 4 serves picker 4.
picker 5 is currently not served
robot 2 serves picker 6.
total costs: 6.0


6.0

In [70]:
# Import packages.
import cvxpy as cp
import numpy as np

# Generate a random non-trivial linear program.
m = 3
n = 2
np.random.seed(1)
#s0 = np.array([1,0,1])
#lamb0 = np.array([2,1,1])
#s0 = np.maximum(s0, 0)
#x0 = np.array([0,0])
#A = np.random.randn(m, n)
distance_matrix = np.array([[0, 1, 2, 3], [1, 0, 1, 2], [2, 1, 0, 1], [3, 2, 1, 0]])
allocation = cp.Variable((4,2))
example_allocation = np.array([[1, 0, 0, 0], [0, 0, 1, 0]]).T
distance_pickers = np.array([[0, 1, 2, 3], [0, 1, 2, 3], [2, 1, 0, 1]])

print(cp.sum(example_allocation, axis=0).value)
print(cp.sum(distance_pickers@example_allocation).value)
#print(example_allocation.T @ example_allocation)


#b = np.array([100, 100, 100])
# Define and solve the CVXPY problem.
x = cp.Variable(n)
prob = cp.Problem(cp.Minimize(cp.sum(distance_pickers@allocation)),
                 [cp.sum(allocation, axis=0) == 1, cp.sum(allocation, axis=1) == 1])
prob.solve(verbose=False)

# Print result.
print("\nThe optimal value is", prob.value)
print("A solution x is")
print(x.value)
print("A dual solution is")
print(prob.constraints[0].dual_value)


[1. 1.]
6.0

The optimal value is inf
A solution x is
None
A dual solution is
None


In [393]:
# Import packages.
import cvxpy as cp
import numpy as np

# Generate a random non-trivial linear program.
m = 3
n = 2
np.random.seed(1)
#s0 = np.array([1,0,1])
#lamb0 = np.array([2,1,1])
#s0 = np.maximum(s0, 0)
#x0 = np.array([0,0])
#A = np.random.randn(m, n)
# picker places = 0, 2, 3, 4
# robots at 1, 3, 4
# r1: 
def get_costs_assignment(p, r, urgency=None):
    # repeat the positions as columns and rows to prepare for diff
    r_mat = np.tile(r, (p.size,1))
    #robot_places_mat = np.append(robot_places_mat, np.zeros((picker_places.,)), axis=1)
    p_mat = np.tile(p, (r.size,1)).transpose()

    #display('robot_places_mat: ', r_mat)
    #display('picker_places_mat: ', p_mat)

    # calculate costs simply as 
    cost = np.abs(p_mat - r_mat)
    if urgency is not None:
        cost = cost * np.tile(urgency, (r.size,1)).transpose()

    # add zero costs for unassigned relations (make it square)
    #cost = np.append(cost, 0*np.ones((p.size, p.size - r.size)), axis=1)
    #display('costs (rows: pickers, columns: robots): ', cost)

    return cost

pickers = np.array([1, 2, 3, 9, 9, 9])
urgency = [0.5, 0, 0, 1, 1, 1]
robots = np.array([3, 9])

cost = get_costs_assignment(pickers, robots, urgency=urgency)
#cost = np.array([[0, 1, 2, 3], [1, 0, 1, 2], [3, 2, 1, 0], [0,0,0,0]])
allocation = cp.Variable(cost.shape, integer=False)
print(cost)
b = np.ones((3,3))
c = np.zeros((3,3))
#b = np.array([100, 100, 100])
# Define and solve the CVXPY problem.
prob = cp.Problem(cp.Minimize(cp.sum(cp.diag(cost.T@allocation))),
                [
                    cp.sum(allocation,axis=1) == 1,
                    cp.sum(allocation,axis=0) >= 1,
                    allocation >= 0
                ]
                )
prob.solve(verbose=False)

# Print result.
print("\nThe optimal value is", prob.value)
print("A solution is")
print(np.round(allocation.value * 100))
print(cost.T @ allocation.value)
print('residual costs: %f%%' % (100* prob.value / np.sum(cost)))
#print("A dual solution is")
#print(prob.constraints[0].dual_value)


[[2 8]
 [1 7]
 [0 6]
 [6 0]
 [6 0]
 [6 0]]

The optimal value is 3.000000000062516
A solution is
[[100.   0.]
 [100.   0.]
 [100.   0.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]]
[[3.0000000e+00 1.8000000e+01]
 [2.1000000e+01 3.6466995e-11]]
residual costs: 7.142857%


In [362]:
np.sum(cost), np.sum(cost.T@allocation.value)

(58, 58.0)

In [7]:
import numpy as np
from itertools import combinations
from math import inf

def state_space_size(n_r=2, n_p=10):
    return np.math.factorial(n_p) / np.math.factorial(n_r) / np.math.factorial(n_p - n_r)

def get_all_states(n_r=2, n_p=10):
    return combinations(range(n_p), n_r)

def get_all_allocations(n_r=2, n_p=10):
    for p1 in range(n_p):
        

def get_cost(p, r):
    # repeat the positions as columns and rows to prepare for diff
    r_mat = np.tile(r, (p.size,1))
    #robot_places_mat = np.append(robot_places_mat, np.zeros((picker_places.,)), axis=1)
    p_mat = np.tile(p, (r.size,1)).transpose()

    #display('robot_places_mat: ', r_mat)
    #display('picker_places_mat: ', p_mat)

    # calculate costs simply as 
    cost = np.abs(p_mat - r_mat)
    print(cost)
    allocation = np.array([[1, 1, 1, 1, 0, 0, 0, 0],[0, 0, 0, 0, 1, 1, 1, 1]])
    print(allocation)
    print('allocation costs:', np.diag(np.matmul(allocation, cost)))

    # add zero costs for unassigned relations (make it square)
    #cost = np.append(cost, np.zeros((p.size, p.size - r.size)), axis=1)
    #display('costs (rows: pickers, columns: robots): ', cost)

    return np.sum(cost)

def search_lowest_costs(p, r):
    # repeat the positions as columns and rows to prepare for diff
    r_mat = np.tile(r, (p.size,1))
    #robot_places_mat = np.append(robot_places_mat, np.zeros((picker_places.,)), axis=1)
    p_mat = np.tile(p, (r.size,1)).transpose()

    #display('robot_places_mat: ', r_mat)
    #display('picker_places_mat: ', p_mat)

    # calculate costs simply as 
    cost = np.abs(p_mat - r_mat)

    # add zero costs for unassigned relations (make it square)
    #cost = np.append(cost, np.zeros((p.size, p.size - r.size)), axis=1)
    #display('costs (rows: pickers, columns: robots): ', cost)

    return np.sum(cost)

def search(states, pickers):
    min_costs = inf
    best = None
    for s in states:
        sa =  np.array(s)
        #print(sa)
        costs = get_cost(pickers, sa)
        print(s, costs)
        if costs < min_costs:
            min_costs = costs
            best = s
    return best, min_costs

number_of_robots = 2
number_of_places = 10

print(state_space_size(number_of_robots, number_of_places))
c = get_all_states(number_of_robots, number_of_places)
#print(list(c))


pickers = np.array([1, 1, 1, 1, 9, 9, 9, 9])
state = np.array([1, 8])

get_cost(pickers, state)

#search(c, pickers)



45.0
[[0 7]
 [0 7]
 [0 7]
 [0 7]
 [8 1]
 [8 1]
 [8 1]
 [8 1]]
[[1 1 1 1 0 0 0 0]
 [0 0 0 0 1 1 1 1]]
allocation costs: [0 4]


64

In [69]:
list(combinations(range(8),7))
3**3

27